### Setup
Import dependencies and initialize shared demo variables used throughout the notebook.

In [ ]:
"""Jupyter notebook demo for pymatviz widgets."""
# /// script
# dependencies = [
#     "pymatgen>=2024.1.1",
#     "ase>=3.22.0",
#     "phonopy>=2.20.0",
# ]
# ///

# %%
import itertools
from typing import Final

import numpy as np
from ase.build import bulk, molecule
from ipywidgets import GridBox, Layout
from phonopy.structure.atoms import PhonopyAtoms
from pymatgen.core import Composition, Lattice, Structure

import pymatviz as pmv


np_rng = np.random.default_rng(seed=0)

### Convex Hull Widget
Build a small Li-Fe-O phase diagram and visualize stable and unstable entries.

In [ ]:
# %% Convex Hull — compute stability from PhaseDiagram entries
from pymatgen.analysis.phase_diagram import PDEntry, PhaseDiagram


phase_diag = PhaseDiagram(
    [
        PDEntry(Composition("Li"), -1.9),
        PDEntry(Composition("Fe"), -4.2),
        PDEntry(Composition("O"), -3.0),
        PDEntry(Composition("Li2O"), -14.3),
        PDEntry(Composition("Fe2O3"), -25.5),
        PDEntry(Composition("LiFeO2"), -18.0),
        PDEntry(Composition("FeO"), -8.5),
    ]
)

pmv.ConvexHullWidget(entries=phase_diag, style="height: 500px;")

### Structure Widget
Render wurtzite GaN as an interactive 3D crystal structure with bonds.

In [ ]:
# %% Structure Widget — wurtzite GaN (hexagonal, more interesting than cubic)
struct = Structure(
    lattice=Lattice.hexagonal(3.19, 5.19),
    species=["Ga", "Ga", "N", "N"],
    coords=[
        [1 / 3, 2 / 3, 0],
        [2 / 3, 1 / 3, 0.5],
        [1 / 3, 2 / 3, 0.375],
        [2 / 3, 1 / 3, 0.875],
    ],
)

pmv.StructureWidget(structure=struct, show_bonds=True, style="height: 400px;")

### Brillouin Zone Widget
Show the reciprocal-space Brillouin zone for the structure above.

In [ ]:
# %% Brillouin Zone — hexagonal BZ from GaN
pmv.BrillouinZoneWidget(structure=struct, show_vectors=True, style="height: 400px;")

### XRD Widget
Compute and display an XRD pattern for rutile TiO2.

In [ ]:
# %% XRD Pattern — rutile TiO2 (tetragonal, richer peak pattern than cubic Si)
from pymatgen.analysis.diffraction.xrd import XRDCalculator


tio2_struct = Structure(
    Lattice.tetragonal(4.594, 2.959),
    ["Ti", "Ti", "O", "O", "O", "O"],
    [
        [0, 0, 0],
        [0.5, 0.5, 0.5],
        [0.305, 0.305, 0],
        [0.695, 0.695, 0],
        [0.195, 0.805, 0.5],
        [0.805, 0.195, 0.5],
    ],
)
xrd_pattern = XRDCalculator().get_pattern(tio2_struct)

pmv.XrdWidget(patterns=xrd_pattern, style="height: 350px;")

### Trajectory Widget
Generate a short perturbed Fe trajectory and visualize it with force-related metadata.

In [ ]:
# %% Trajectory Widget — expanding lattice with energy/force properties
trajectory = []
base_struct = Structure(
    lattice=Lattice.cubic(3.0),
    species=("Fe", "Fe"),
    coords=((0, 0, 0), (0.5, 0.5, 0.5)),
)

for idx in range(n_steps := 20):
    struct_frame = base_struct.perturb(distance=0.2).copy()
    energy = n_steps / 2 - idx * np_rng.random()
    np.fill_diagonal(dist_max := struct_frame.distance_matrix, np.inf)
    trajectory.append(
        {"structure": struct_frame, "energy": energy, "force_max": 1 / dist_max.min()}
    )

pmv.TrajectoryWidget(
    trajectory=trajectory,
    display_mode="structure+scatter",
    show_force_vectors=True,
    style="height: 600px;",
)

### Scatter Plot Widget
Dual-axis comparison of `sin(x)` and `cos(x)` on shared x-values.

In [ ]:
# %% ScatterPlot Widget — dual-axis trigonometric curves
scatter_series = [
    {
        "label": "sin(x)",
        "x": np.linspace(0, 6.0, 60).tolist(),
        "y": np.sin(np.linspace(0, 6.0, 60)).tolist(),
    },
    {
        "label": "cos(x)",
        "x": np.linspace(0, 6.0, 60).tolist(),
        "y": np.cos(np.linspace(0, 6.0, 60)).tolist(),
        "y_axis": "y2",
    },
]

pmv.ScatterPlotWidget(
    series=scatter_series,
    x_axis={"label": "x"},
    y_axis={"label": "sin(x)"},
    y2_axis={"label": "cos(x)", "color": "#ff7f0e"},
    display={"x_grid": True, "y_grid": True},
    legend={"position": "top-right"},
    style="height: 420px;",
)

### Bar Plot Widget
Grouped bars compare two model score series over the same sample index axis.

In [ ]:
# %% BarPlot Widget — grouped comparison bars
bar_series = [
    {"label": "Model A", "x": [0, 1, 2], "y": [4.2, 5.1, 4.8]},
    {"label": "Model B", "x": [0, 1, 2], "y": [3.9, 4.6, 5.2]},
]

pmv.BarPlotWidget(
    series=bar_series,
    mode="grouped",
    x_axis={"label": "Sample index"},
    y_axis={"label": "Score"},
    display={"y_grid": True},
    style="height: 360px;",
)

### Histogram Widget
Overlayed histograms summarize the value distributions of the two trigonometric series.

In [ ]:
# %% Histogram Widget — distribution overlay for scatter data
histogram_series = [
    {
        "label": scatter_series[0]["label"],
        "x": scatter_series[0]["x"],
        "y": scatter_series[0]["y"],
    },
    {
        "label": scatter_series[1]["label"],
        "x": scatter_series[1]["x"],
        "y": scatter_series[1]["y"],
    },
]

pmv.HistogramWidget(
    series=histogram_series,
    bins=20,
    mode="overlay",
    x_axis={"label": "Value"},
    y_axis={"label": "Count"},
    style="height: 360px;",
)

### Band Structure Widget
Load realistic phonon band structure data from test fixtures.

In [ ]:
# %% Band Structure Widget — fixture-based demo
import json

from monty.io import zopen
from monty.json import MontyDecoder

from pymatviz.utils.testing import TEST_FILES


phonon_fixture_path = f"{TEST_FILES}/phonons/mp-2758-Sr4Se4-pbe.json.xz"
with zopen(phonon_fixture_path, mode="rt") as file:
    phonon_doc = json.loads(file.read(), cls=MontyDecoder)

band_data = phonon_doc.phonon_bandstructure

pmv.BandStructureWidget(band_structure=band_data, style="height: 400px;")

### DOS Widget
Use matching phonon DOS data from the same fixture document.

In [ ]:
# %% DOS Widget — fixture-based demo
dos_data = phonon_doc.phonon_dos

pmv.DosWidget(dos=dos_data, style="height: 400px;")

### Bands + DOS Widget
Combine band structure and density of states into one coordinated view.

In [ ]:
# %% Combined Bands + DOS
pmv.BandsAndDosWidget(band_structure=band_data, dos=dos_data, style="height: 500px;")

### Composition Widgets Grid
Compare multiple compositions across pie, bar, and bubble display modes.

In [ ]:
# %% Composition Widget — grid of compositions x modes
comps = (
    "Fe2 O3",
    Composition("Li P O4"),
    dict(Co=20, Cr=20, Fe=20, Mn=20, Ni=20),
    dict(Ti=20, Zr=20, Nb=20, Mo=20, V=20),
)
modes = ("pie", "bar", "bubble")
size = 100
children = [
    pmv.CompositionWidget(
        composition=comp,
        mode=mode,
        style=f"width: {(1 + (mode == 'bar')) * size}px; height: {size}px;",
    )
    for comp, mode in itertools.product(comps, modes)
]
layout = Layout(
    grid_template_columns=f"repeat({len(modes)}, auto)",
    grid_gap="2em 4em",
    padding="2em",
)
GridBox(children=children, layout=layout)

GridBox(children=(<pymatviz.widgets.composition.CompositionWidget object at 0x12d80f7a0>, <pymatviz.widgets.co…

### Remote Trajectory Widget
Load and visualize a trajectory file directly from a remote URL.

In [ ]:
# %% Render remote ASE trajectory file
matterviz_traj_dir_url: Final = (
    "https://github.com/janosh/matterviz/raw/6288721042/src/site/trajectories"
)

file_name = "Cr0.25Fe0.25Co0.25Ni0.25-mace-omat-qha.xyz.gz"
pmv.TrajectoryWidget(
    data_url=f"{matterviz_traj_dir_url}/{file_name}",
    display_mode="structure+scatter",
    show_force_vectors=True,
    force_vector_scale=0.5,
    force_vector_color="#ff4444",
    show_bonds=True,
    bonding_strategy="nearest_neighbor",
    style="height: 600px;",
)

### ASE Atoms MIME Rendering
Display an ASE bulk structure via pymatviz MIME auto-rendering.

In [ ]:
# %% ASE Atoms — auto-rendered via MIME type recognition
ase_atoms = bulk("Al", "fcc", a=4.05)
ase_atoms *= (2, 2, 2)
ase_atoms

### ASE Molecule MIME Rendering
Display a simple ASE molecule via pymatviz MIME auto-rendering.

In [ ]:
# %% ASE molecule
ase_molecule = molecule("H2O")
ase_molecule.center(vacuum=3.0)
ase_molecule

### PhonopyAtoms MIME Rendering
Display a PhonopyAtoms structure via pymatviz MIME auto-rendering.

In [ ]:
# %% PhonopyAtoms — auto-rendered via MIME type recognition
PhonopyAtoms(
    symbols=["Na", "Cl"],
    positions=[[0.0, 0.0, 0.0], [0.5, 0.5, 0.5]],
    cell=[[4, 0, 0], [0, 4, 0], [0, 0, 4]],
)